Для отклика на вакансию предложено следующее тестовое задание:

Есть SQL база с таблицами:
1) Users(userId, age)
2) Purchases (purchaseId, userId, itemId, date)
3) Items (itemId, price).


Написать SQL запросы для расчета следующих метрик:

А) какую сумму в среднем в месяц тратят:
- пользователи в возрастном диапазоне от 18 до 25 лет включительно
- пользователи в возрастном диапазоне от 26 до 35 лет включительно

Б) в каком месяце года выручка от пользователей в возрастном диапазоне 35+ самая большая

В) какой товар обеспечивает дает наибольший вклад в выручку за последний год

Г) топ-3 товаров по выручке и их доля в общей выручке за любой год

РЕШЕНИЕ:

Наше приложение для работы будет использовать SQLAlchemy и sqlite

In [2]:
import random
import time
import datetime

from sqlalchemy import Table, Column, Integer, String, MetaData, create_engine, func, or_
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy.schema import ForeignKey

engine = create_engine('sqlite://')
Session = sessionmaker(bind=engine)
metadata = MetaData()
Base = declarative_base()
session = Session()

Базовый класс наших моделей

In [3]:
class DB():
    session = Session()
    
    def __init__(self, kwargs = False):
        if kwargs:
            try:
                f = self.session.query(self._sa_instance_state.class_).filter_by(**kwargs).one_or_none()
                self.id = f.id
            except Exception as e:
                print(e)


    def save(self):
        try:
            self.session.commit()
        except Exception as e:
            print(e)

            
    def add(self):
        if self.id:
            return self.id
        self.session.add(self)
        self.save()
        return self.id

Описание моделей:

In [4]:
class Users(Base, DB):
    __tablename__ = 'users'
    id = Column(Integer, primary_key=True)
    age = Column(Integer, nullable=False)
    
    
    def add_purchase(self, itemid):
        purchase = Purchases()
        purchase.userid = self.id
        purchase.itemid = itemid
        return purchase.add()

In [5]:
class Purchases(Base, DB):
    __tablename__ = 'purchases'
    id = Column(Integer, primary_key=True)
    userid = Column(Integer, ForeignKey('users.id'))
    itemid = Column(Integer, ForeignKey('items.id'))
    date = Column(String, nullable=False)

    
    def add(self):
        #to generate a date in the interval of the last 3 years
        present_time = int(time.time())
        past_tense = present_time - 94694400
        date = random.randint(past_tense, present_time)
        self.date = datetime.datetime.utcfromtimestamp(date).strftime("%Y-%m")
        return  super().add()
        

In [6]:
class Items(Base, DB):
    __tablename__ = 'items'
    id = Column(Integer, primary_key=True)
    price = Column(Integer, nullable=False)

Генерим DB из metadata:

In [7]:
Base.metadata.create_all(engine)

Наполняем DB позициями в количестве M штук (стоимость позиции от 500 до 1000 у.е.):

In [8]:
M = 100

for _ in range(M):
    item = Items()
    item.price = random.randint(500, 1000)
    item.add()

Наполняем  N пользователями и случайным количеством покупок (от 1 до 50) на каждого:

In [9]:
N = 1000
for _ in range(N):
    user = Users()
    user.age = random.randint(14, 50)
    user.add()
    for _ in range(random.randint(1, 50)): #в покупке одна случайная позиция
        user.add_purchase(random.randint(1, M))

DB готова, приступаем к data analysis:

А) какую сумму в среднем в месяц тратят:
- пользователи в возрастном диапазоне от 18 до 25 лет включительно
- пользователи в возрастном диапазоне от 26 до 35 лет включительно

In [10]:
def average_costs(age):
    q1 = session.query(func.avg(Items.price)).join(Purchases, Purchases.itemid == Items.id).join(Users, Purchases.userid == Users.id).filter(Users.age.in_((age))).group_by(Purchases.date)
    return session.query(func.avg(q1))

Вычисляем средние траты в месяц пользователей в возрастном диапазоне от 18 до 25 лет включительно за 3 последних года:

In [11]:
query = average_costs((18,25))
print(query)

SELECT avg((SELECT avg(items.price) AS avg_2 
FROM items JOIN purchases ON purchases.itemid = items.id JOIN users ON purchases.userid = users.id 
WHERE users.age IN ([POSTCOMPILE_age_1]) GROUP BY purchases.date)) AS avg_1


/opt/conda/lib/python3.8/site-packages/sqlalchemy/sql/coercions.py:307: SAWarning: implicitly coercing SELECT object to scalar subquery; please use the .scalar_subquery() method to produce a scalar subquery.
  util.warn(


In [12]:
print(int(query[0][0]))

656


Вычисляем средние траты в месяц пользователей в возрастном диапазоне от 18 до 25 лет включительно за 3 последних года:

In [13]:
query = average_costs((26,35))
print(query)

SELECT avg((SELECT avg(items.price) AS avg_2 
FROM items JOIN purchases ON purchases.itemid = items.id JOIN users ON purchases.userid = users.id 
WHERE users.age IN ([POSTCOMPILE_age_1]) GROUP BY purchases.date)) AS avg_1


In [14]:
print(int(query[0][0]))

737


Б) В каком месяце года выручка от пользователей в возрастном диапазоне 35+ самая большая:

In [15]:
q1  = session.query(func.sum(Items.price)).join(Purchases, Purchases.itemid == Items.id).group_by(Purchases.date)
query = session.query(func.max(q1), Purchases.date).join(Users, Users.age >= 35)

In [16]:
print(query)

SELECT max((SELECT sum(items.price) AS sum_1 
FROM items JOIN purchases ON purchases.itemid = items.id GROUP BY purchases.date)) AS max_1, purchases.date AS purchases_date 
FROM purchases JOIN users ON users.age >= ?


In [17]:
print(query[0][1])

2019-05


В) какой товар дает наибольший вклад в выручку за последний год:

In [18]:
past_tense = int(time.time()) - 31536000
last_year = datetime.datetime.utcfromtimestamp(past_tense).strftime("%Y")

In [41]:
q1  = session.query(func.sum(Items.price)).join(Purchases, Purchases.itemid == Items.id).filter(Purchases.date >= last_year).group_by(Items.id)
query = session.query(func.max(q1), Items.id)

In [42]:
print(q1)

SELECT sum(items.price) AS sum_1 
FROM items JOIN purchases ON purchases.itemid = items.id 
WHERE purchases.date >= ? GROUP BY items.id


In [43]:
print(query.all())

[(110160, 1)]
